In [ ]:
pip install fuzzymatcher

In [ ]:
aid_fuzzy = aid_data1[['Report_Type','c5_aid','Date_Of_Occurrence','c13_aid','c23_aid','c24_aid','c95_aid','N_Number','Remarks']]

sdr_fuzzy = sdr_data1[['Report_Type','c14_sdr','c18_sdr','Date_Of_Occurrence','c130_sdr','c140_sdr','c100_sdr','c250_sdr','c260_sdr','c332_sdr','N_Number','Remarks']]

eon_fuzzy = eon_data1[['Report_Type','Id','Date_Of_Occurrence','N_Number','Aircraft Type','Remarks']]

In [ ]:
aid_fuzzy.columns.to_list()

In [ ]:
sdr_fuzzy.columns.to_list()

In [ ]:
eon_fuzzy.columns.to_list()

In [ ]:
aid_fuzzy['Aircraft_Type'] = aid_fuzzy['c23_aid'] + ' '+ aid_fuzzy['c24_aid']
aid_fuzzy['Aircraft_Type'] = aid_fuzzy['Aircraft_Type'].str.strip().str.replace('\s+', ' ', regex=True).astype(str)
aid_fuzzy['Aircraft_Type']

aid_fuzzy = aid_fuzzy.drop(['c23_aid','c24_aid'],axis=1)

aid_fuzzy.rename(columns={'Date_Of_Occurrence': 'Date_Of_Occurrence_aid', 'N_Number': 'N_Number_aid','Aircraft_Type':'Aircraft_Type_aid','Remarks':'Remarks_aid'}, inplace=True)

In [ ]:
sdr_fuzzy['Aircraft_Type'] = sdr_fuzzy['c130_sdr'] + ' '+ sdr_fuzzy['c140_sdr']
sdr_fuzzy['Aircraft_Type'] = sdr_fuzzy['Aircraft_Type'].str.strip().str.replace('\s+', ' ', regex=True).astype(str)
sdr_fuzzy['Aircraft_Type']

sdr_fuzzy['Defective_Part_Description'] = sdr_fuzzy['c100_sdr'] + ' ' + sdr_fuzzy['c250_sdr'] + ' ' + sdr_fuzzy['c260_sdr']
sdr_fuzzy['Defective_Part_Description'] = sdr_fuzzy['Defective_Part_Description'].str.strip().str.replace('\s+', ' ', regex=True).astype(str)
sdr_fuzzy['Defective_Part_Description']

sdr_fuzzy = sdr_fuzzy.drop(['c100_sdr','c130_sdr','c140_sdr','c250_sdr','c260_sdr'],axis=1)

sdr_fuzzy['c18_sdr'] = sdr_fuzzy['c18_sdr'].str.replace('\W', '')

sdr_fuzzy.rename(columns={'Date_Of_Occurrence': 'Date_Of_Occurrence_sdr', 'N_Number': 'N_Number_sdr','Aircraft_Type':'Aircraft_Type_sdr','Remarks':'Remarks_sdr'}, inplace=True)

In [ ]:
import fuzzymatcher

In [ ]:
fil = eon_fuzzy['N_Number'].str.contains(r'^[A-Z]',na=False)
eon_fuzzy = eon_fuzzy[~fil]

eon_fuzzy.reset_index(drop=True,inplace=True)

eon_fuzzy.rename(columns={'Date_Of_Occurrence': 'Date_Of_Occurrence_eon', 'N_Number': 'N_Number_eon','Aircraft Type':'Aircraft_Type_eon','Remarks':'Remarks_eon'}, inplace=True)

eon_fuzzy

In [ ]:
left_on = ["Date_Of_Occurrence_aid", "Aircraft_Type_aid", "N_Number_aid"]

right_on = ["Date_Of_Occurrence_eon", "Aircraft_Type_eon", "N_Number_eon"]

matched_results = fuzzymatcher.fuzzy_left_join(aid_fuzzy,
                                            eon_fuzzy,
                                            left_on,
                                            right_on,
                                            left_id_col='c5_aid',
                                            right_id_col='Id')

cols = ["best_match_score", 'c5_aid', 'Id', "Date_Of_Occurrence_aid", "Date_Of_Occurrence_eon", "Aircraft_Type_aid", "Aircraft_Type_eon",
        "N_Number_aid", "N_Number_eon","Remarks_aid","Remarks_eon"]

matched_results[cols].sort_values(by=['best_match_score'], ascending=False).head(10)

In [ ]:
left_on = ["Date_Of_Occurrence_eon", "Aircraft_Type_eon", "N_Number_eon"]

right_on = ["Date_Of_Occurrence_sdr", "Aircraft_Type_sdr", "N_Number_sdr"]

matched_results = fuzzymatcher.fuzzy_left_join(eon_fuzzy,
                                            sdr_fuzzy,
                                            left_on,
                                            right_on,
                                            left_id_col='Id',
                                            right_id_col='c18_sdr')

cols = ["best_match_score", 'Id', 'c18_sdr', "Date_Of_Occurrence_eon", "Date_Of_Occurrence_sdr", "Aircraft_Type_eon", "Aircraft_Type_sdr",
        "N_Number_eon", "N_Number_sdr","Remarks_eon","Remarks_sdr"]

matched_results[cols].sort_values(by=['best_match_score'], ascending=False).head(10)